***Description of problem.***
With the Covid-19 pandemic, there is a need for patients to have access to hospitals for treatment. This has been challenging as the demand for medical care, as measured by the number of patients, has exceeded the supply of hospital beds in qualifying facilities in many locations.

Our scenario involves 9 counties where patients reside, and 14 hospitals. In order to provide additional capacity for Covid patients, hospitals can set up temporary facilities on their premises. For the purpose of this analysis, this will increase capacity by up to 100 patients at each hospital. We need to allocate all patients to a specific hospital, and do so while minimizing the overall cost for transportation and extra capacity.

***Describe solution for problem***
We built a model to allocate patients to hospitals based on their locations, the demand and available capacity, including extra facilities co-located with existing hospitals. This model is a mixed integer (this includes both binary and continuous variables) linear program built within a Python framework.

The location of all patients are based on the centroid coordinates of their county of residence. Each county has a specific level of demand for medical care, and each hospital has a capacity for care that it can provide in their regular facilities. We can measure the distance between each county centroid and hospital location in order to calculate the travel cost, which is $5 per mile.

There is an objective function to calculate the total cost, which consists of three pieces. One is the travel cost mentioned previously. The others are a fixed cost to set up extra capacity, regardless of how much extra capacity is used; and a variable cost that is based on the number of patients using the extra capacity.

The optimization problem is set up with three constraints, in order to minimize the total cost. These constraints align the total demand with total hospital capacity; ensures that the aggregate extra capacity can accommodate the demand; and sets a maximum extra capacity level in each hospital.

Once the optimization problem was solved, after all 14 hospitals had their regular facilities allocated, 3 hospitals were identified for expanded capacity. Of these, 2 used all of the expanded capacity available and 1 used only enough to ensure the remaining patients were allocated. The optimal (i.e. minimal) final cost was calculated as about $1,140,129.

In [49]:
# There are Covid patients from several counties. These patients need to be assigned to hospitals
# which have varied levels of capacity, and each have 100 units of extra capacity outside their regular facilities.

import pulp as p
from pulp import LpBinary,value,lpSum,LpStatus

In [50]:
# identification of nnumerous counties
COUNTIES = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

# identification of numerous hospitals
HOSPITALS = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8', 'h9', 'h10', 'h11', 'h12', 'h13', 'h14']

In [51]:
# locations of county centroids that are associated with Covid patients

county_coord_list = {
'c1': [1.0, 1.5],
'c2': [3.0, 1.0],
'c3': [5.5, 1.5],
'c4': [1.0, 4.5],
'c5': [3.0, 3.5],
'c6': [5.5, 4.5],
'c7': [1.0, 8.0],
'c8': [3.0, 6.0],
'c9': [4.5, 8.0]}

In [52]:
# demand level representing the number of Covid patients in each county

county_demand_list = {
'c1': 351,
'c2': 230,
'c3': 529,
'c4': 339,
'c5': 360,
'c6': 527,
'c7': 469,
'c8': 234,
'c9': 500}

In [53]:
# locations of hospitals

hosp_coord_list = {
'h1': [1.0, 2.0],
'h2': [2.5, 1.0],
'h3': [5.0, 1.0],
'h4': [6.5, 3.5],
'h5': [1.0, 5.0],
'h6': [3.0, 4.0],
'h7': [5.0, 4.0],
'h8': [6.5, 5.5],
'h9': [1.0, 8.5],
'h10': [1.5, 9.5],
'h11': [8.5, 6.0],
'h12': [5.0, 8.0],
'h13': [3.0, 9.0],
'h14': [6.0, 9.0]}

In [54]:
# capacity of regular facilities at each hospital

hosp_cap_list = {
'h1': 281,
'h2': 187,
'h3': 200,
'h4': 223,
'h5': 281,
'h6': 281,
'h7': 222,
'h8': 200,
'h9': 250,
'h10': 125,
'h11': 187,
'h12': 300,
'h13': 300,
'h14': 243}

In [55]:
# constant for travel cost
cost_per_mile = 5
# constant for population share with Covid
covid_rate = .1
# constant for extra capacity in hospitals outside regular facilities
hosp_extra_cap = 100

In [56]:
# constant for fixed cost to set up Covid units in hospital 
fixed_cost = 200000
# constant for cost per Covid patient in hospital
cost_per_patient = 2000

In [87]:
# sum of total Covid patients, i.e. demand
demand_values = county_demand_list.values()
print(sum(demand_values))

3539


In [88]:
# sum of total Covid treatment capacity in regular hospital facilities
cap_values = hosp_cap_list.values()
print(sum(cap_values))

3280


In [67]:
import pandas as pd
import numpy as np

# create data frame for matrix with distances between each county centroid and hospital
distance = pd.DataFrame(np.arange(126.000).reshape((9,14)),
                   index=COUNTIES,
                   columns=HOSPITALS)

In [70]:
# populate data frame with distances between county centroids and hospitals
for i in county_coord_list:
    for j in hosp_coord_list:
        distance[j][i] = float(np.sqrt((county_coord_list[i][0] - hosp_coord_list[j][0])**2 + (county_coord_list[i][1] - hosp_coord_list[j][1])**2))

In [71]:
distance

,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,h11,h12,h13,h14
c1,0.500000,1.581139,4.031129,5.852350,3.500000,3.201562,4.716991,6.800735,7.000000,8.015610,8.746428,7.632169,7.762087,9.013878
c2,2.236068,0.500000,2.000000,4.301163,4.472136,3.000000,3.605551,5.700877,7.762087,8.631338,7.433034,7.280110,8.000000,8.544004
c3,4.527693,3.041381,0.707107,2.236068,5.700877,3.535534,2.549510,4.123106,8.321658,8.944272,5.408327,6.519202,7.905694,7.516648
c4,2.500000,3.807887,5.315073,5.590170,0.500000,2.061553,4.031129,5.590170,4.000000,5.024938,7.648529,5.315073,4.924429,6.726812
c5,2.500000,2.549510,3.201562,3.500000,2.500000,0.500000,2.061553,4.031129,5.385165,6.184658,6.041523,4.924429,5.500000,6.264982
c6,5.147815,4.609772,3.535534,1.414214,4.527693,2.549510,0.707107,1.414214,6.020797,6.403124,3.354102,3.535534,5.147815,4.527693
c7,6.000000,7.158911,8.062258,7.106335,3.000000,4.472136,5.656854,6.041523,0.500000,1.581139,7.762087,4.000000,2.236068,5.099020
c8,4.472136,5.024938,5.385165,4.301163,2.236068,2.000000,2.828427,3.535534,3.201562,3.807887,5.500000,2.828427,3.000000,4.242641
c9,6.946222,7.280110,7.017834,4.924429,4.609772,4.272002,4.031129,3.201562,3.535534,3.354102,4.472136,0.500000,1.802776,1.802776


In [73]:
distance['h1']['c1']

0.5

In [74]:
county_demand_list['c1']

351

In [75]:
# define problem as having the objective to identify a resource allocation solution with the minimum cost
prob=p.LpProblem('Covid',p.LpMinimize)

In [76]:
# variable for # of assigned patients from counties to hospitals
ASSIGNMENTS=[(i,j) for i in COUNTIES for j in HOSPITALS]
num_patients = p.LpVariable.dicts('Service',ASSIGNMENTS,0)

In [77]:
# binary variable for whether or not each hospital will have additional capacity added
expand = p.LpVariable.dicts('UseLocation',HOSPITALS,0,1,LpBinary)

In [78]:
# define additional capacity values for each hospital
add_capac = p.LpVariable.dicts('ExtraCapacity',HOSPITALS,0)

In [79]:
# objective function includes travel cost, fixed cost (if capacity is added) and variable cost (per unit of capacity added)
prob += p.lpSum(cost_per_mile*distance[j][i]*num_patients[(i,j)] for i in COUNTIES for j in HOSPITALS) \
+ p.lpSum(fixed_cost * expand[j] for j in HOSPITALS) + p.lpSum(cost_per_patient * add_capac[j] for j in HOSPITALS)

In [80]:
# constraint to ensure that total demand from patients is met by the total capacity in hospitals
for i in COUNTIES:
    prob += p.lpSum(num_patients[(i,j)] for j in HOSPITALS) == county_demand_list[i]

In [82]:
# constraint to ensure that the total demand from patients is no greater than the total capacity in hospitals, 
# including the extra capacity
for j in HOSPITALS:
    prob += p.lpSum(num_patients[(i,j)] for i in COUNTIES) <= hosp_cap_list[j] + add_capac[j]

In [83]:
# constraint to ensure that additional capacity allocated per hospital is no greater than the extra capacity available
# for those hospitals where extra capacity is used
for j in HOSPITALS:
    prob += add_capac[j] <= hosp_extra_cap * expand[j]

In [84]:
# solve optimization problem
prob.solve()

1

In [85]:
# print status of optimization problem
print('Current Status',LpStatus[prob.status])

Current Status Optimal


In [86]:
# for optimal model, display whether or not extra capacity is needed at each hospital
for j in HOSPITALS:
    print(j, expand[j].varValue)

h1 1.0
h2 0.0
h3 1.0
h4 0.0
h5 1.0
h6 0.0
h7 0.0
h8 0.0
h9 0.0
h10 0.0
h11 0.0
h12 0.0
h13 0.0
h14 0.0


In [89]:
# after running model, allocation of extra capacity per hospital
for j in HOSPITALS:
    print(j, add_capac[j].varValue)

h1 100.0
h2 0.0
h3 100.0
h4 0.0
h5 59.0
h6 0.0
h7 0.0
h8 0.0
h9 0.0
h10 0.0
h11 0.0
h12 0.0
h13 0.0
h14 0.0


In [90]:
# display final allocation of each county based set of patients to hospitals
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, '=', v.varValue)

ExtraCapacity_h1 = 100.0
ExtraCapacity_h3 = 100.0
ExtraCapacity_h5 = 59.0
Service_('c1',_'h1') = 351.0
Service_('c2',_'h1') = 30.0
Service_('c2',_'h2') = 187.0
Service_('c2',_'h7') = 13.0
Service_('c3',_'h3') = 300.0
Service_('c3',_'h4') = 223.0
Service_('c3',_'h7') = 6.0
Service_('c4',_'h5') = 339.0
Service_('c5',_'h5') = 1.0
Service_('c5',_'h6') = 281.0
Service_('c5',_'h7') = 78.0
Service_('c6',_'h11') = 187.0
Service_('c6',_'h14') = 15.0
Service_('c6',_'h7') = 125.0
Service_('c6',_'h8') = 200.0
Service_('c7',_'h10') = 125.0
Service_('c7',_'h13') = 94.0
Service_('c7',_'h9') = 250.0
Service_('c8',_'h12') = 28.0
Service_('c8',_'h13') = 206.0
Service_('c9',_'h12') = 272.0
Service_('c9',_'h14') = 228.0
UseLocation_h1 = 1.0
UseLocation_h3 = 1.0
UseLocation_h5 = 1.0


In [91]:
# final cost as determined by optimal model
print(value(prob.objective))

1140128.7632103662
